# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティション向けのエージェント作成プロセスを示しています。本ノートブックの主な目的は、20の質問形式のゲームをプレイするためのAIエージェントの構築と、最終的な提出物（`submission.tar.gz`）を生成することです。

### 問題の概要
「20の質問」ゲームでは、ある言葉を当てるために限られた質問への答え（「はい」または「いいえ」）に基づいて、情報を収集し、最終的にその言葉を推測することが求められます。ノートブックでは、質問者と回答者の役割を担うエージェントを開発することが核心です。

### 使用している手法
1. **エージェントの設計**: `Gemma`というライブラリを利用して、カジュアルな言語モデルを用いたエージェントを構築します。これには、予め定義されたプロンプトや例を用いて、エージェントがターンごとに自然な対話を行えるように設定されています。

2. **フォーマッティング**: `GemmaFormatter`クラスを使用して、ユーザーとモデルの対話をフォーマットします。ユーザーの質問やモデルの応答を適切に管理し、トークンでの開始・終了マーカーも含まれます。

3. **エージェントクラスの実装**: `GemmaQuestionerAgent`および`GemmaAnswererAgent`の二つのクラスを実装し、それぞれ質問を行うため・応答するためのメソッドが定義されています。

4. **モデルの初期化**: PyTorchを利用して、Gemmaモデルの構成を取得し、モデルの重みをロードすることで、エージェントが適切に動作する準備を整えます。

### 使用しているライブラリ
- **PyTorch**: モデルの構築や学習を行うために使用。
- **Gemma**: 特に言語モデル用に設計されたライブラリで、カジュアルな応答生成に用いられます。
- **sentencepiece**: トークナイザーを用いて、テキストの前処理を支援します。
- **immutabledict**: 不変の辞書を提供し、ハッシュ可能なデータ構造を支援します。

最終的に、このノートブックは、必要なライブラリをインストールし、エージェントの実装を確立し、テスト後にファイルを圧縮してコンペティションに提出するための準備を整えます。

---


# 用語概説 
以下は、Jupyter Notebookの内容を基に、機械学習・深層学習の初心者がつまずきそうな専門用語の解説です。重要な用語や一般的な知識は除外しています。

1. **Gemma**: 特定の NLP モデルのフレームワークで、質問応答タスク向けに設計されています。このノートブックでは、Gemma の PyTorch 実装を使用して、20の質問ゲームのエージェントを作成しています。

2. **エージェント (Agent)**: 状態を持ち、特定のタスクを実行するプログラムやモデルのこと。この場合、質問者と回答者の役割を持つエージェントについてです。

3. **few-shot examples**: モデルが学習するために使う、少数の具体的な例のこと。新しいタスクに対するモデルの適応を助けます。このケースでは、20の質問ゲームに関するいくつかの例を示しています。

4. **トークナイザー**: 自然言語テキストを、モデルが扱える形式に変換するツールまたはプロセス。トークンとは、単語やサブワードの単位を指します。

5. **量子化 (Quantization)**: モデルのサイズを減らすために、フロート型の重みを整数型に変換する技術。メモリ使用量を削減し、推論速度を向上させるために利用されます。

6. **ケラフレーム (Checkpoint)**: モデルの学習中の状態を保存したもの。トレーニングの途中から再開できるようにするためのスナップショットです。

7. **dyanmic programming**: 計算を分割し、部分問題の結果を記録して再利用するアルゴリズム技法。その性質により、パフォーマンスを向上させることができます。

8. **リセット (Reset)**: 状態や条件を初期の状態に戻すプロセスで、ここではエージェントの状態をクリアして新たに開始することを指します。

9. **教師なし学習 (Unsupervised Learning)**: 明示的なラベルなしでデータを利用して特徴を抽出する手法。特に、データの構造やパターンを見つける際に重要です。

10. **プロンプト (Prompt)**: 言語モデルに対する問い合わせや命令文のこと。特に、このノートブックでは、モデルに対する指示の形式を整える役割を果たします。

11. **セッション (Session)**: モデルが特定のタスクを実行するために必要な状態を保持するプロセス。ここでは、質問や回答のやり取りを状態として保持します。

12. **擬似コード (Pseudocode)**: プログラミング言語の文法に依存せず、アルゴリズムを表現するための説明的なコードを書くこと。特にこのノートブックでのモデルの動作フローを明確にするために使用されます。

これらの解説は、初心者が特に混乱しやすい用語や概念に焦点を当てています。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

This notebook illustrates the agent creation process for the **LLM 20 Questions**. Running this notebook produces a `submission.tar.gz` file. You may submit this file directly from the **Submit to competition** heading to the right. Alternatively, from the notebook viewer, click the *Output* tab then find and download `submission.tar.gz`. Click **Submit Agent** at the upper-left of the competition homepage to upload your file and make your submission. 

</div>
<div class="column-right">

# 日本語訳

このノートブックは、**LLM 20 Questions**のエージェント作成プロセスを示しています。このノートブックを実行すると、`submission.tar.gz`ファイルが生成されます。このファイルは、右側の**コンペティションに提出**の見出しから直接提出できます。または、ノートブックビューアから*出力*タブをクリックし、`submission.tar.gz`を見つけてダウンロードします。競技のホームページの左上の**Agent Submit**をクリックして、ファイルをアップロードし、提出を完了してください。



</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%bash
cd /kaggle/working
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir /kaggle/working/submission/lib/gemma/
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/
```

</div>
<div class="column-right">

# 日本語訳

```python
%%bash
cd /kaggle/working
# 必要なライブラリをインストールします。-qオプションは出力を抑制します。
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
# GemmaのPyTorch実装をGitHubからクローンします。
git clone https://github.com/google/gemma_pytorch.git > /dev/null
# Gemmaのライブラリ用のディレクトリを作成します。
mkdir /kaggle/working/submission/lib/gemma/
# Gemmaのクローンからライブラリを移動します。
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/
```

</div>
</details>

In [ ]:
%%bash
cd /kaggle/working
# 必要なライブラリをインストールします。-qオプションは出力を抑制します。
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
# GemmaのPyTorch実装をGitHubからクローンします。
git clone https://github.com/google/gemma_pytorch.git > /dev/null
# Gemmaのライブラリ用のディレクトリを作成します。
mkdir /kaggle/working/submission/lib/gemma/
# Gemmaのクローンからライブラリを移動します。
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%writefile submission/main.py
# Setup
import os
import sys

# **IMPORTANT:** Set up your system path like this to make your code work
# both in notebooks and in the simulations environment.
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")

import contextlib
import os
import sys
from pathlib import Path

import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

if os.path.exists(KAGGLE_AGENT_PATH):
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/7b-it-quant/2")
else:
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/7b-it-quant/2"

# Prompt Formatting
import itertools
from typing import Iterable


class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    def __repr__(self):
        return self._state

    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self

    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self

    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self

    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self

    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self

    def reset(self):
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self


# Agent Definitions
import re


@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """Set the default torch dtype to the given dtype."""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)


class GemmaAgent:
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._variant = variant
        self._device = torch.device(device)
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)

        print("Initializing model")
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b()
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")
        model_config.quant = "quant" in variant

        with _set_default_tensor_type(model_config.get_dtype()):
            model = GemmaForCausalLM(model_config)
            ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-{variant}.ckpt')
            model.load_weights(ckpt_path)
            self.model = model.to(self._device).eval()

    def __call__(self, obs, *args):
        self._start_session(obs)
        prompt = str(self.formatter)
        response = self._call_llm(prompt)
        response = self._parse_response(response, obs)
        print(f"{response=}")
        return response

    def _start_session(self, obs: dict):
        raise NotImplementedError

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        if sampler_kwargs is None:
            sampler_kwargs = {
                'temperature': 0.01,
                'top_p': 0.1,
                'top_k': 1,
        }
        response = self.model.generate(
            prompt,
            device=self._device,
            output_len=max_new_tokens,
            **sampler_kwargs,
        )
        return response

    def _parse_keyword(self, response: str):
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        if match is None:
            keyword = ''
        else:
            keyword = match.group().lower()
        return keyword

    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError


def interleave_unequal(x, y):
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]


class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user("Let's play 20 Questions. You are playing the role of the Questioner.")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        if obs.turnType == 'ask':
            self.formatter.user("Please ask a yes-or-no question.")
        elif obs.turnType == 'guess':
            self.formatter.user("Now guess the keyword. Surround your guess with double asterisks.")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None:
                question = "Is it a person?"
            else:
                question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            return guess
        else:
            raise ValueError("Unknown turn type:", obs.turnType)


class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user(f"Let's play 20 Questions. You are playing the role of the Answerer. The keyword is {obs.keyword} in the category {obs.category}.")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"The question is about the keyword {obs.keyword} in the category {obs.category}. Give yes-or-no answer and surround your answer with double asterisks, like **yes** or **no**.")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'


# Agent Creation
system_prompt = "You are an AI assistant designed to play the 20 Questions game. In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner. The keyword is a specific person, place, or thing."

few_shot_examples = [
    "Let's play 20 Questions. You are playing the role of the Questioner. Please ask your first question.",
    "Is it a person?", "**no**",
    "Is is a place?", "**yes**",
    "Is it a country?", "**yes** Now guess the keyword.",
    "**France**", "Correct!",
]


# **IMPORTANT:** Define agent as a global so you only have to load
# the agent you need. Loading both will likely lead to OOM.
agent = None


def get_agent(name: str):
    global agent
    
    if agent is None and name == 'questioner':
        agent = GemmaQuestionerAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    elif agent is None and name == 'answerer':
        agent = GemmaAnswererAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    assert agent is not None, "Agent not initialized."

    return agent


def agent_fn(obs, cfg):
    if obs.turnType == "ask":
        response = get_agent('questioner')(obs)
    elif obs.turnType == "guess":
        response = get_agent('questioner')(obs)
    elif obs.turnType == "answer":
        response = get_agent('answerer')(obs)
    if response is None or len(response) <= 1:
        return "yes"
    else:
        return response
```

</div>
<div class="column-right">

# 日本語訳

```python
%%writefile submission/main.py
# セットアップ
import os
import sys

# **重要:** このようにシステムパスを設定して、ノートブックとシミュレーション環境の両方でコードが動作するようにします。
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")

import contextlib
import os
import sys
from pathlib import Path

import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

# モデルの重みパスを設定します。
if os.path.exists(KAGGLE_AGENT_PATH):
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/7b-it-quant/2")
else:
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/7b-it-quant/2"

# プロンプトのフォーマット
import itertools
from typing import Iterable


class GemmaFormatter:
    _start_token = '<start_of_turn>' # ターン開始のトークン
    _end_token = '<end_of_turn>' # ターン終了のトークン

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    def __repr__(self):
        return self._state

    def user(self, prompt):
        self._state += self._turn_user.format(prompt) # ユーザープロンプトを追加
        return self

    def model(self, prompt):
        self._state += self._turn_model.format(prompt) # モデルプロンプトを追加
        return self

    def start_user_turn(self):
        self._state += f"{self._start_token}user\n" # ユーザーのターンを開始
        return self

    def start_model_turn(self):
        self._state += f"{self._start_token}model\n" # モデルのターンを開始
        return self

    def end_turn(self):
        self._state += f"{self._end_token}\n" # ターンを終了
        return self

    def reset(self):
        self._state = "" # 状態をリセット
        if self._system_prompt is not None:
            self.user(self._system_prompt) # システムプロンプトを追加
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user') # few-shot例を適用
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters) # 交互に使用するためのサイクルを作成
        for fmt, turn in zip(formatters, turns):
            fmt(turn) # フォーマッタを適用
        return self


# エージェントの定義
import re


@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """与えられたdtypeをデフォルトのtorch dtypeとして設定します。"""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float) # デフォルトのdtypeを元に戻す


class GemmaAgent:
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._variant = variant
        self._device = torch.device(device) # デバイスの設定
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples) # フォーマッタのインスタンスを作成

        print("モデルを初期化しています")
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b() # モデル設定の取得
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model") # トークナイザーのパスを設定
        model_config.quant = "quant" in variant # 量子化の設定

        with _set_default_tensor_type(model_config.get_dtype()):
            model = GemmaForCausalLM(model_config) # モデルのインスタンスを作成
            ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-{variant}.ckpt') # チェックポイントのパスを設定
            model.load_weights(ckpt_path) # 重みをロード
            self.model = model.to(self._device).eval() # モデルをデバイスに移動し、評価モードにする

    def __call__(self, obs, *args):
        self._start_session(obs) # セッションの開始
        prompt = str(self.formatter) # フォーマッタからプロンプトを取得
        response = self._call_llm(prompt) # LLMに呼び出す
        response = self._parse_response(response, obs) # レスポンスを解析
        print(f"{response=}")
        return response

    def _start_session(self, obs: dict):
        raise NotImplementedError

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        if sampler_kwargs is None:
            sampler_kwargs = {
                'temperature': 0.01,
                'top_p': 0.1,
                'top_k': 1,
        }
        response = self.model.generate(
            prompt,
            device=self._device, # 使用するデバイス
            output_len=max_new_tokens, # 生成するトークン数
            **sampler_kwargs,
        )
        return response

    def _parse_keyword(self, response: str):
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response) # キーワードを抽出
        if match is None:
            keyword = ''
        else:
            keyword = match.group().lower() # 小文字に変換
        return keyword

    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError


def interleave_unequal(x, y):
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ] # 不等長のリストを交互に統合する関数


class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs) # 親クラスの初期化

    def _start_session(self, obs):
        self.formatter.reset() # フォーマッタをリセット
        self.formatter.user("20の質問をプレイしましょう。あなたは質問者の役割を担います。") # ユーザーの初期メッセージ
        turns = interleave_unequal(obs.questions, obs.answers) # 質問と回答を交互に統合
        self.formatter.apply_turns(turns, start_agent='model') # 過去のターンを適用
        if obs.turnType == 'ask':
            self.formatter.user("はいかいいえで答えられる質問をしてください。") # 質問時
        elif obs.turnType == 'guess':
            self.formatter.user("キーワードを推測してください。推測は二重アスタリスクで囲んでください。") # 推測時
        self.formatter.start_model_turn() # モデルのターンを開始

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', '')) # 質問を抽出
            if match is None:
                question = "人ですか？" # デフォルト質問
            else:
                question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response) # 推測を解析
            return guess
        else:
            raise ValueError("不明なターンタイプ:", obs.turnType) # エラー処理


class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs) # 親クラスの初期化

    def _start_session(self, obs):
        self.formatter.reset() # フォーマッタをリセット
        self.formatter.user(f"20の質問をプレイしましょう。あなたは回答者の役割を担います。キーワードは{obs.keyword}で、カテゴリは{obs.category}です。") # ユーザーの初期メッセージ
        turns = interleave_unequal(obs.questions, obs.answers) # 質問と回答を交互に統合
        self.formatter.apply_turns(turns, start_agent='user') # 過去のターンを適用
        self.formatter.user(f"この質問はキーワード{obs.keyword}のカテゴリ{obs.category}に関するものです。はいかいいえで答え、答案は二重アスタリスクで囲んでください。")
        self.formatter.start_model_turn() # モデルのターンを開始

    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response) # 応答を解析
        return 'yes' if 'yes' in answer else 'no' # yesまたはnoを返す


# エージェントの作成
system_prompt = "あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、回答者がキーワードを思い浮かべ、質問者のはいまたはいいえの質問に対して応答します。キーワードは特定の人、場所、物です。"

few_shot_examples = [
    "20の質問をプレイしましょう。あなたは質問者の役割を担います。最初の質問をしてください。",
    "それは人ですか？", "**いいえ**",
    "それは場所ですか？", "**はい**",
    "それは国ですか？", "**はい** ではキーワードを推測してください。",
    "**フランス**", "正解！",
]


# **重要:** エージェントをグローバルに定義します。そうすることで、必要なエージェントだけをロードできます。両方をロードすると、OOM（メモリ不足）が発生する可能性があります。
agent = None


def get_agent(name: str):
    global agent
    
    if agent is None and name == 'questioner':
        agent = GemmaQuestionerAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    elif agent is None and name == 'answerer':
        agent = GemmaAnswererAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    assert agent is not None, "エージェントが初期化されていません。"

    return agent


def agent_fn(obs, cfg):
    if obs.turnType == "ask":
        response = get_agent('questioner')(obs) # 質問者エージェントを呼び出す
    elif obs.turnType == "guess":
        response = get_agent('questioner')(obs) # 質問者エージェントを呼び出す
    elif obs.turnType == "answer":
        response = get_agent('answerer')(obs) # 回答者エージェントを呼び出す
    if response is None or len(response) <= 1:
        return "はい" # デフォルトの応答
    else:
        return response
```

</div>
</details>

In [ ]:
%%writefile submission/main.py
# セットアップ
import os
import sys

# **重要:** このようにシステムパスを設定して、ノートブックとシミュレーション環境の両方でコードが動作するようにします。
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")

import contextlib
import os
import sys
from pathlib import Path

import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

# モデルの重みパスを設定します。
if os.path.exists(KAGGLE_AGENT_PATH):
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/7b-it-quant/2")
else:
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/7b-it-quant/2"

# プロンプトのフォーマット
import itertools
from typing import Iterable


class GemmaFormatter:
    _start_token = '<start_of_turn>' # ターン開始のトークン
    _end_token = '<end_of_turn>' # ターン終了のトークン

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    def __repr__(self):
        return self._state

    def user(self, prompt):
        self._state += self._turn_user.format(prompt) # ユーザープロンプトを追加
        return self

    def model(self, prompt):
        self._state += self._turn_model.format(prompt) # モデルプロンプトを追加
        return self

    def start_user_turn(self):
        self._state += f"{self._start_token}user\n" # ユーザーのターンを開始
        return self

    def start_model_turn(self):
        self._state += f"{self._start_token}model\n" # モデルのターンを開始
        return self

    def end_turn(self):
        self._state += f"{self._end_token}\n" # ターンを終了
        return self

    def reset(self):
        self._state = "" # 状態をリセット
        if self._system_prompt is not None:
            self.user(self._system_prompt) # システムプロンプトを追加
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user') # few-shot例を適用
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters) # 交互に使用するためのサイクルを作成
        for fmt, turn in zip(formatters, turns):
            fmt(turn) # フォーマッタを適用
        return self


# エージェントの定義
import re


@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """与えられたdtypeをデフォルトのtorch dtypeとして設定します。"""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float) # デフォルトのdtypeを元に戻す


class GemmaAgent:
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._variant = variant
        self._device = torch.device(device) # デバイスの設定
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples) # フォーマッタのインスタンスを作成

        print("モデルを初期化しています")
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b() # モデル設定の取得
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model") # トークナイザーのパスを設定
        model_config.quant = "quant" in variant # 量子化の設定

        with _set_default_tensor_type(model_config.get_dtype()):
            model = GemmaForCausalLM(model_config) # モデルのインスタンスを作成
            ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-{variant}.ckpt') # チェックポイントのパスを設定
            model.load_weights(ckpt_path) # 重みをロード
            self.model = model.to(self._device).eval() # モデルをデバイスに移動し、評価モードにする

    def __call__(self, obs, *args):
        self._start_session(obs) # セッションの開始
        prompt = str(self.formatter) # フォーマッタからプロンプトを取得
        response = self._call_llm(prompt) # LLMに呼び出す
        response = self._parse_response(response, obs) # レスポンスを解析
        print(f"{response=}")
        return response

    def _start_session(self, obs: dict):
        raise NotImplementedError

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        if sampler_kwargs is None:
            sampler_kwargs = {
                'temperature': 0.01,
                'top_p': 0.1,
                'top_k': 1,
        }
        response = self.model.generate(
            prompt,
            device=self._device, # 使用するデバイス
            output_len=max_new_tokens, # 生成するトークン数
            **sampler_kwargs,
        )
        return response

    def _parse_keyword(self, response: str):
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response) # キーワードを抽出
        if match is None:
            keyword = ''
        else:
            keyword = match.group().lower() # 小文字に変換
        return keyword

    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError


def interleave_unequal(x, y):
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ] # 不等長のリストを交互に統合する関数


class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs) # 親クラスの初期化

    def _start_session(self, obs):
        self.formatter.reset() # フォーマッタをリセット
        self.formatter.user("20の質問をプレイしましょう。あなたは質問者の役割を担います。") # ユーザーの初期メッセージ
        turns = interleave_unequal(obs.questions, obs.answers) # 質問と回答を交互に統合
        self.formatter.apply_turns(turns, start_agent='model') # 過去のターンを適用
        if obs.turnType == 'ask':
            self.formatter.user("はいかいいえで答えられる質問をしてください。") # 質問時
        elif obs.turnType == 'guess':
            self.formatter.user("キーワードを推測してください。推測は二重アスタリスクで囲んでください。") # 推測時
        self.formatter.start_model_turn() # モデルのターンを開始

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', '')) # 質問を抽出
            if match is None:
                question = "人ですか？" # デフォルト質問
            else:
                question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response) # 推測を解析
            return guess
        else:
            raise ValueError("不明なターンタイプ:", obs.turnType) # エラー処理


class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs) # 親クラスの初期化

    def _start_session(self, obs):
        self.formatter.reset() # フォーマッタをリセット
        self.formatter.user(f"20の質問をプレイしましょう。あなたは回答者の役割を担います。キーワードは{obs.keyword}で、カテゴリは{obs.category}です。") # ユーザーの初期メッセージ
        turns = interleave_unequal(obs.questions, obs.answers) # 質問と回答を交互に統合
        self.formatter.apply_turns(turns, start_agent='user') # 過去のターンを適用
        self.formatter.user(f"この質問はキーワード{obs.keyword}のカテゴリ{obs.category}に関するものです。はいかいいえで答え、答案は二重アスタリスクで囲んでください。")
        self.formatter.start_model_turn() # モデルのターンを開始

    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response) # 応答を解析
        return 'yes' if 'yes' in answer else 'no' # yesまたはnoを返す


# エージェントの作成
system_prompt = "あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、回答者がキーワードを思い浮かべ、質問者のはいまたはいいえの質問に対して応答します。キーワードは特定の人、場所、物です。"

few_shot_examples = [
    "20の質問をプレイしましょう。あなたは質問者の役割を担います。最初の質問をしてください。",
    "それは人ですか？", "**いいえ**",
    "それは場所ですか？", "**はい**",
    "それは国ですか？", "**はい** ではキーワードを推測してください。",
    "**フランス**", "正解！",
]


# **重要:** エージェントをグローバルに定義します。そうすることで、必要なエージェントだけをロードできます。両方をロードすると、OOM（メモリ不足）が発生する可能性があります。
agent = None


def get_agent(name: str):
    global agent
    
    if agent is None and name == 'questioner':
        agent = GemmaQuestionerAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    elif agent is None and name == 'answerer':
        agent = GemmaAnswererAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    assert agent is not None, "エージェントが初期化されていません。"

    return agent


def agent_fn(obs, cfg):
    if obs.turnType == "ask":
        response = get_agent('questioner')(obs) # 質問者エージェントを呼び出す
    elif obs.turnType == "guess":
        response = get_agent('questioner')(obs) # 質問者エージェントを呼び出す
    elif obs.turnType == "answer":
        response = get_agent('answerer')(obs) # 回答者エージェントを呼び出す
    if response is None or len(response) <= 1:
        return "はい" # デフォルトの応答
    else:
        return response

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!apt install pigz pv > /dev/null
```

</div>
<div class="column-right">

# 日本語訳

```python
# pigzとpvをインストールします。出力を抑制します。
!apt install pigz pv > /dev/null
```

</div>
</details>

In [ ]:
# pigzとpvをインストールします。出力を抑制します。
!apt install pigz pv > /dev/null

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/7b-it-quant/2
```

</div>
<div class="column-right">

# 日本語訳

```python
# submissionディレクトリ内のすべてのファイルをtar.gz圧縮します。
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/7b-it-quant/2
```

</div>
</details>

In [ ]:
# submissionディレクトリ内のすべてのファイルをtar.gz圧縮します。
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/7b-it-quant/2